# Parquet to Dask
load a parquet dataset into as dask cluster

Once the client connects to the dask cluster we can check for package consistency by calling `dask_client.get_versions(check=True)`.  Scheduler and worker
nodes are guaranteed to have the same package versions in `mlrun` since mismatches would likely cause serialization failures.  The client (in this notebook or in some other process) may have different versions and this mismatch is **less likely** to cause issues.  The following installs can be performed to ensure this mismatch becomes **unlikely** (as of this writing). **This may need to be done periodically as the dockerfiles refresh, unless you specifiy an image tag.**

    !python -m pip install --upgrade pip
    !python -m pip uninstall -y cmake
    !python -m pip install cmake
    !python -m pip install scikit-build

    !python -m pip install lz4
    !python -m pip install git+https://github.com/Blosc/python-blosc.git@v1.8.3
    !python -m pip install cloudpickle==1.2.2

    !python -m pip install numpy==1.17.4

## clean up
delete stale pods

In [ ]:
# !mlrun clean -p -r

In [1]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [2]:
# replace with your app node ip
mlrun.mlconf.remote_host = '18.223.249.136' 

## parameters


In [3]:
FUNCTION           = 'parquet_to_dask'                                # python function file name
DESCRIPTION        = 'load parquet dataset into a dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                      # custom image
JOB_KIND           = 'dask'                                           

TASK_NAME          = 'user-task-parq-to-dask'                         # whatever

ARTIFACTS_PATH     = '/User/repos/dask/artifacts'
YAML_PATH          = '/User/repos/dask/yaml'

DATA_PATH          = '/User/repos/dask/dataset/partitions'
DOWNSAMPLE         = 0.15
PARTITION_COLS     = ['Year', 'Month']
PARTITION_COLS     = None

DASK_SHARDS        = 8
DASK_THREADS_PER   = 8
MEMORY_LIMIT       = '5GB'

DASK_KEY           = 'airlines'

## load and configure function

In [4]:
FUNC_PY = '/User/repos/dask/parquet-to-dask-airlines.py'
FUNC_YAML = '/User/repos/dask/yaml/parquet-to-dask-airlines.yaml'

#### some of the following cells can be commented out if this notebook has already been run.

In [5]:
# load function from a local Python file
parq2dask = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)

parq2dask.spec.remote = True
parq2dask.spec.replicas = DASK_SHARDS 
parq2dask.spec.max_replicas = DASK_SHARDS
parq2dask.spec.service_type = 'NodePort'
parq2dask.spec.image_pull_policy = 'Always'
parq2dask.spec.build.image = IMAGE

parq2dask.export(FUNC_YAML)
# parq2dask = mlrun.import_function(FUNC_YAML)

parq2dask.apply(mlrun.mount_v3io())
parq2dask.deploy()

# create and run the task
parq_to_dask_task = mlrun.NewTask(
    TASK_NAME, 
    handler=FUNCTION,  
    params={
        'parquet_url'      : DATA_PATH,
        'sample'           : DOWNSAMPLE,
        'shards'           : DASK_SHARDS,
        'threads_per'      : DASK_THREADS_PER,
        'memory_limit'     : MEMORY_LIMIT,
        'dask_key'         : DASK_KEY,
        'target_path'      : ARTIFACTS_PATH})

# run
rn = parq2dask.run(parq_to_dask_task)

[mlrun] 2020-02-09 22:46:10,162 function spec saved to path: /User/ecolab/yaml/parquet-to-dask-airlines.yaml
[mlrun] 2020-02-09 22:46:10,230 starting remote build, image: yjbds/mlrun-daskboost:dev
[mlrun] 2020-02-09 22:46:10,283 starting run user-task-parq-to-dask uid=2eab3b2dfad042479b40efba3b96e538  -> http://mlrun-api:8080
[mlrun] 2020-02-09 22:46:11,588 saving function: parquet-to-dask-airlines, tag: latest
[mlrun] 2020-02-09 22:46:19,429 using remote dask scheduler (mlrun-parquet-to-dask-airlines-f8e4d16d-3) at: 18.223.249.136:30602
[mlrun] 2020-02-09 22:46:19,431 remote dashboard (node) port: 18.223.249.136:31734
[mlrun] 2020-02-09 22:46:19,431 found cluster...
[mlrun] 2020-02-09 22:46:19,432 <Client: 'tcp://10.233.64.49:8786' processes=0 threads=0, memory=0 B>
[mlrun] 2020-02-09 22:48:24,731 column header ['Year' 'Month' 'DayofMonth' 'DayOfWeek' 'CRSDepTime' 'UniqueCarrier'
 'ArrDelay' 'Origin' 'Dest' 'Distance']
[mlrun] 2020-02-09 22:48:25,850 log artifact scheduler at /User/ec

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...96e538,0,Feb 09 22:46:11,completed,user-task-parq-to-dask,kind=daskowner=adminhost=jupyter-1-56795c755c-bss6b,,parquet_url=/User/ecolab/dataset/partitionssample=0.15shards=8threads_per=8memory_limit=5GBdask_key=airlinestarget_path=/User/ecolab/artifacts,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2eab3b2dfad042479b40efba3b96e538  , !mlrun logs 2eab3b2dfad042479b40efba3b96e538 
[mlrun] 2020-02-09 22:48:25,927 run executed, status=completed


### the scheduler artifact can be shared

In [6]:
print(rn.metadata.to_yaml())

uid: 2eab3b2dfad042479b40efba3b96e538
name: user-task-parq-to-dask
project: ''
labels:
  kind: dask
  owner: admin
  host: jupyter-1-56795c755c-bss6b
iteration: 0



In [7]:
rn.metadata.uid

'2eab3b2dfad042479b40efba3b96e538'

In [8]:
rn.outputs['scheduler']

'/User/ecolab/artifacts/scheduler.json'

#### What's the scheduler address?

In [9]:
import json
json.load(open(rn.outputs['scheduler']))

{'type': 'Scheduler',
 'id': 'Scheduler-4c7d1e0e-4b69-4abc-aaf4-d5e3fc791ff4',
 'address': 'tcp://10.233.64.49:8786',
 'services': {},
 'workers': {'tcp://10.233.64.52:45749': {'type': 'Worker',
   'id': 'tcp://10.233.64.52:45749',
   'host': '10.233.64.52',
   'resources': {},
   'local_directory': '/worker-yf6emgxk',
   'name': 'tcp://10.233.64.52:45749',
   'nthreads': 1,
   'memory_limit': 4109136640,
   'last_seen': 1581288505.696887,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 92479488,
    'time': 1581288505.6854692,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'num_fds': 22,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://10.233.64.52:43634'},
  'tcp://10.233.64.53:43116': {'type': 'Worker',
   'id': 'tcp://10.233.64.53:43116',
   'host': '10.233.64.53',
   'resources': {},
   'local_directory': '/worker-azf329ti',
   'name': 'tcp://10.233.64.

### notebook cluster

Let's load the scheduler file into a cluster in this notebook using our MLRun artifact:

In [10]:
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

#client = Client(scheduler_file=rn.outputs['scheduler'])

#df = client.get_dataset(DASK_KEY)

### create a component 'on the fly' to summarise the table

The nice thing about having a dask cluster loaded with all your data is that you can write _quick and dirty_ jobs either in your notebook, a local file, or a github repo. Here we use a local file:

In [12]:
# write up function in local directory
summ = mlrun.new_function(command='/User/repos/dask/describe.py', 
                          image=IMAGE,
                          kind='job')

summ.spec.build.image = IMAGE

summ.export('/User/repos/dask/yaml/describe.yaml')

summ.apply(mlrun.mount_v3io())

summ.deploy(skip_deployed=True, with_mlrun=False)

# create the task
summ_task = mlrun.NewTask(
    'user-task-my-sum', 
    handler='table_summary',  
    params={
        'dask_key'   :  DASK_KEY,
        'dask_client': 'scheduler.json',
        'target_path':  ARTIFACTS_PATH,
        'name'       : 'table-summary.csv',
        'key'        : 'table-summary'})

# run
rn2 = summ.run(summ_task)

rn2.outputs

[mlrun] 2020-02-09 22:48:26,004 function spec saved to path: /User/ecolab/yaml/describe.yaml
[mlrun] 2020-02-09 22:48:26,011 starting run user-task-my-sum uid=12881df967bd48ffb7b59ebd6cbcba30  -> http://mlrun-api:8080
[mlrun] 2020-02-09 22:48:26,072 Job is running in the background, pod: user-task-my-sum-kzz74
[mlrun] 2020-02-09 22:58:01,663 log artifact table-summary at /User/ecolab/artifacts/table-summary.csv, size: None, db: Y

[mlrun] 2020-02-09 22:58:01,711 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...bcba30,0,Feb 09 22:48:36,completed,describe,host=user-task-my-sum-kzz74kind=jobowner=admin,,dask_client=scheduler.jsondask_key=airlineskey=table-summaryname=table-summary.csvtarget_path=/User/ecolab/artifacts,,table-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run 12881df967bd48ffb7b59ebd6cbcba30  , !mlrun logs 12881df967bd48ffb7b59ebd6cbcba30 
[mlrun] 2020-02-09 22:58:08,026 run executed, status=completed


{'table-summary': '/User/ecolab/artifacts/table-summary.csv'}

### tests

In [13]:
import pandas as pd
pd.read_csv(rn2.outputs['table-summary'])

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,Distance
0,2.779537e+06,2.779537e+06,2.779537e+06,2.779537e+06,2.779537e+06,2.721146e+06,2.774851e+06
1,1.998624e+03,6.553820e+00,1.572539e+01,3.939387e+00,1.334742e+03,7.030874e+00,7.015807e+02
2,6.226768e+00,3.444456e+00,8.786220e+00,1.990051e+00,4.761106e+02,3.064334e+01,5.509279e+02
3,1.987000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-6.880000e+02,1.100000e+01
4,1.995000e+03,5.000000e+00,1.100000e+01,3.000000e+00,1.120000e+03,-4.000000e+00,4.050000e+02
5,2.003000e+03,9.000000e+00,2.000000e+01,5.000000e+00,1.530000e+03,4.000000e+00,6.960000e+02
6,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.050000e+03,7.750000e+01,2.486000e+03
7,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,1.429000e+03,4.983000e+03
